In [ ]:
# Reference
# https://yurimkoo.github.io/python/2019/09/14/connect-db-with-python.html

import pandas as pd
import requests
import pymysql
import json
import os
from dotenv import load_dotenv
from sqlalchemy import create_engine

In [ ]:
# Load api key & RDS info (endpoint="rds 엔드포인트", dbname="db 이름", username="마스터 사용자 이름", password="rds 비밀번호")
load_dotenv()
api_key = os.environ.get("API_KEY")
endpoint = os.environ.get("AWS_endpoint")
dbname = os.environ.get('AWS_dbname')
username = os.environ.get("AWS_username")
password = os.environ.get("AWS_password")

In [ ]:
# Get the list of available tournaments
url = "https://api.pubg.com/tournaments"

header = {
  "Authorization": api_key,
  "Accept": "application/vnd.api+json"
}
r = requests.get(url, headers=header)
tournament_list = r.json()

# Tournaments_id 값 추출
tournament_id = pd.DataFrame(tournament_list["data"])["id"].values

print(f"{len(tournament_id)} 개의 tournament_id 조회 완료\n")
print(f"tournament_id 상위 5개 항목 조회 결과: {tournament_id[:5]}")

In [ ]:
from functions.pubgapi import get_match_info

for i in range(len(tournament_id)):
    cur_id = tournament_id[i]
    cur_match_info = get_match_info(api_key, cur_id)

In [ ]:
# tournament_id를 모두 불러와 각 id별 match_info 생성
for val in tournament_id:

  # 해당 tournament_id를 불러오기 위한 url 작성
  url = f"https://api.pubg.com/tournaments/{val}"
  header = {"Authorization": api_key, "Accept": "application/vnd.api+json", "charset": "UTF-8"}
  
  # requests.get 함수를 이용하여 tournament_list를 json으로 호출
  r = requests.get(url, headers=header)
  current_tournament = r.json()

  # 모든 match_id와 created_at을 dict으로 출력
  matchId_dict = {match['attributes']['createdAt']: match['id'] for match in current_tournament['included']}

  # matchId_dict를 createdAt로 정렬 후 DataFrame으로 변환
  cur_tournament_matchId_df = pd.DataFrame(sorted(matchId_dict.items(), key=lambda x: x[0]), columns=['createdAt', 'matchId'])